# Torch Neural Network Integration with custom integration function

## Libraries imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import math

from mpmath import polylog, exp

## Model definition

### Neural Network implementation

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Sigmoid()
        self.output_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [3]:
def train_model(model, criterion, optimizer, x_train, y_train, epochs):
    for epoch in range(epochs):
        predictions = model(x_train)
        loss = criterion(predictions, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.6f}')

### NN custom integration function

In [4]:
def get_NN_analytical_integral(alpha, beta, b1, b2, weights_w1, weights_w2):

    def compute_phi_j(b1_j, w1_j, alpha, beta):
        term_alpha = polylog(1, -exp(-b1_j - w1_j * alpha))
        term_beta = polylog(1, -exp(-b1_j - w1_j * beta))
        return term_alpha - term_beta

    integral_sum = 0
    for w2, w1, b1_j in zip(weights_w2, weights_w1, b1):
        phi_j = compute_phi_j(b1_j, w1, alpha, beta)
        integral_sum += w2 * ((beta - alpha) + phi_j / w1)
    
    return b2 * (beta - alpha) + integral_sum

### Dataset generator

In [5]:
def generate_data(func, n_samples=100):
    x = torch.linspace(0, 1, n_samples).unsqueeze(1)
    y = func(x)
    return x, y

def cos_func(x):
    return torch.cos(x)

def exp_func(x):
    return torch.exp(x)

## Model application

### Hyperparams

In [6]:
input_size = 1
hidden_size = 10
learning_rate = 0.01
num_epochs = 10000

model_cos = MLP(input_size, hidden_size)
model_exp = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_cos = optim.Adam(model_cos.parameters(), lr=learning_rate)
optimizer_exp = optim.Adam(model_exp.parameters(), lr=learning_rate)

### Dataset generation

In [7]:
x_train_cos, y_train_cos = generate_data(cos_func, 10000)
x_train_exp, y_train_exp = generate_data(exp_func, 10000)

### Model training

In [8]:
train_model(model_cos, criterion, optimizer_cos, x_train_cos, y_train_cos, num_epochs)

Epoch [100/10000], Loss: 0.010741
Epoch [200/10000], Loss: 0.002600
Epoch [300/10000], Loss: 0.001553
Epoch [400/10000], Loss: 0.001421
Epoch [500/10000], Loss: 0.001319
Epoch [600/10000], Loss: 0.001231
Epoch [700/10000], Loss: 0.001154
Epoch [800/10000], Loss: 0.001083
Epoch [900/10000], Loss: 0.001014
Epoch [1000/10000], Loss: 0.000943
Epoch [1100/10000], Loss: 0.000867
Epoch [1200/10000], Loss: 0.000785
Epoch [1300/10000], Loss: 0.000696
Epoch [1400/10000], Loss: 0.000602
Epoch [1500/10000], Loss: 0.000505
Epoch [1600/10000], Loss: 0.000408
Epoch [1700/10000], Loss: 0.000317
Epoch [1800/10000], Loss: 0.000235
Epoch [1900/10000], Loss: 0.000167
Epoch [2000/10000], Loss: 0.000113
Epoch [2100/10000], Loss: 0.000073
Epoch [2200/10000], Loss: 0.000046
Epoch [2300/10000], Loss: 0.000029
Epoch [2400/10000], Loss: 0.000019
Epoch [2500/10000], Loss: 0.000013
Epoch [2600/10000], Loss: 0.000010
Epoch [2700/10000], Loss: 0.000008
Epoch [2800/10000], Loss: 0.000007
Epoch [2900/10000], Loss: 0.0

In [9]:
train_model(model_exp, criterion, optimizer_exp, x_train_exp, y_train_exp, num_epochs)

Epoch [100/10000], Loss: 0.113125
Epoch [200/10000], Loss: 0.053437
Epoch [300/10000], Loss: 0.019245
Epoch [400/10000], Loss: 0.007358
Epoch [500/10000], Loss: 0.005034
Epoch [600/10000], Loss: 0.004415
Epoch [700/10000], Loss: 0.003967
Epoch [800/10000], Loss: 0.003570
Epoch [900/10000], Loss: 0.003214
Epoch [1000/10000], Loss: 0.002893
Epoch [1100/10000], Loss: 0.002599
Epoch [1200/10000], Loss: 0.002328
Epoch [1300/10000], Loss: 0.002077
Epoch [1400/10000], Loss: 0.001843
Epoch [1500/10000], Loss: 0.001624
Epoch [1600/10000], Loss: 0.001420
Epoch [1700/10000], Loss: 0.001231
Epoch [1800/10000], Loss: 0.001056
Epoch [1900/10000], Loss: 0.000895
Epoch [2000/10000], Loss: 0.000750
Epoch [2100/10000], Loss: 0.000620
Epoch [2200/10000], Loss: 0.000506
Epoch [2300/10000], Loss: 0.000407
Epoch [2400/10000], Loss: 0.000324
Epoch [2500/10000], Loss: 0.000256
Epoch [2600/10000], Loss: 0.000200
Epoch [2700/10000], Loss: 0.000157
Epoch [2800/10000], Loss: 0.000123
Epoch [2900/10000], Loss: 0.0

### Extractoin of the parameters of the trained model

In [10]:
def extract_model_params(model):
    b1 = model.input_layer.bias.detach().numpy()
    weights_w1 = model.input_layer.weight.detach().numpy().flatten()
    b2 = model.output_layer.bias.item()
    weights_w2 = model.output_layer.weight.detach().numpy().flatten()

    return b1, weights_w1, b2, weights_w2

In [11]:
b1_cos, weights_w1_cos, b2_cos, weights_w2_cos = extract_model_params(model_cos)

In [12]:
b1_exp, weights_w1_exp, b2_exp, weights_w2_exp = extract_model_params(model_exp)

### Integreation parameters

In [13]:
alpha = 0
beta = 1

### NNI integral calculation

In [14]:
integral_cos = get_NN_analytical_integral(alpha, beta, b1_cos, b2_cos, weights_w1_cos, weights_w2_cos)

In [15]:
print(f"The analytical integral of cos(x) of the trained neural network model is: {float(integral_cos):.6f}")

The analytical integral of cos(x) of the trained neural network model is: 0.841461


In [16]:
integral_exp = get_NN_analytical_integral(alpha, beta, b1_exp, b2_exp, weights_w1_exp, weights_w2_exp)

In [17]:
print(f"The analytical integral of exp(x) of the trained neural network model is: {float(integral_exp):.6f}")

The analytical integral of exp(x) of the trained neural network model is: 1.718283


### True integral for comparison

$\int_{\alpha}^{\beta} dx\{cos(x)\} = sin(\beta) - sin(\alpha) $

$\int_{\alpha}^{\beta} dx\{exp(x)\} = exp(\beta) - exp(\alpha) $

In [18]:
true_integral_cos = np.sin(beta) - np.sin(alpha)
true_integral_exp = np.exp(beta) - np.exp(alpha)

In [19]:
print(f"The true cosinus integral is: {float(true_integral_cos):.6f}")

The true cosinus integral is: 0.841471


In [20]:
print(f"The true exponent integral is: {float(true_integral_exp):.6f}")

The true exponent integral is: 1.718282


## Integration of function 1.1.1:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 1, b = 1 $

In [21]:
a = 1
b = 1

def alpha_func(alpha):
    return alpha ** a * (1 - alpha) ** b

In [22]:
model_alpha = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha.parameters(), lr=learning_rate)

In [23]:
x_train_alpha, y_train_alpha = generate_data(alpha_func, 10000)

In [24]:
train_model(model_alpha, criterion, optimizer_alpha, x_train_alpha, y_train_alpha, num_epochs)

Epoch [100/10000], Loss: 0.005567
Epoch [200/10000], Loss: 0.005509
Epoch [300/10000], Loss: 0.005473
Epoch [400/10000], Loss: 0.005406
Epoch [500/10000], Loss: 0.005250
Epoch [600/10000], Loss: 0.004783
Epoch [700/10000], Loss: 0.003510
Epoch [800/10000], Loss: 0.001313
Epoch [900/10000], Loss: 0.000208
Epoch [1000/10000], Loss: 0.000111
Epoch [1100/10000], Loss: 0.000091
Epoch [1200/10000], Loss: 0.000080
Epoch [1300/10000], Loss: 0.000072
Epoch [1400/10000], Loss: 0.000064
Epoch [1500/10000], Loss: 0.000058
Epoch [1600/10000], Loss: 0.000051
Epoch [1700/10000], Loss: 0.000045
Epoch [1800/10000], Loss: 0.000039
Epoch [1900/10000], Loss: 0.000034
Epoch [2000/10000], Loss: 0.000029
Epoch [2100/10000], Loss: 0.000025
Epoch [2200/10000], Loss: 0.000021
Epoch [2300/10000], Loss: 0.000017
Epoch [2400/10000], Loss: 0.000014
Epoch [2500/10000], Loss: 0.000011
Epoch [2600/10000], Loss: 0.000009
Epoch [2700/10000], Loss: 0.000007
Epoch [2800/10000], Loss: 0.000006
Epoch [2900/10000], Loss: 0.0

In [25]:
b1_alpha, weights_w1_alpha, b2_alpha, weights_w2_alpha = extract_model_params(model_alpha)

In [26]:
integral_alpha = get_NN_analytical_integral(alpha, beta, b1_alpha, b2_alpha, weights_w1_alpha, weights_w2_alpha)

In [27]:
print(f"The analytical integral of exp(x) of the trained neural network model is: {float(integral_alpha):.6f}")

The analytical integral of exp(x) of the trained neural network model is: 0.166672


In [28]:
true_integral_alpha = (1/2 - 1/3)

In [29]:
print(f"The true cosinus integral is: {float(true_integral_alpha):.6f}")

The true cosinus integral is: 0.166667


## Integration of function 1.1.2:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 0, b = 1 $

In [30]:
a = 0
b = 1

In [31]:
model_alpha_2 = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha_2.parameters(), lr=learning_rate)

In [32]:
x_train_alpha_2, y_train_alpha_2 = generate_data(alpha_func, 10000)

In [33]:
train_model(model_alpha_2, criterion, optimizer_alpha, x_train_alpha_2, y_train_alpha_2, num_epochs)

Epoch [100/10000], Loss: 0.040663
Epoch [200/10000], Loss: 0.001378
Epoch [300/10000], Loss: 0.000214
Epoch [400/10000], Loss: 0.000154
Epoch [500/10000], Loss: 0.000110
Epoch [600/10000], Loss: 0.000080
Epoch [700/10000], Loss: 0.000059
Epoch [800/10000], Loss: 0.000045
Epoch [900/10000], Loss: 0.000035
Epoch [1000/10000], Loss: 0.000028
Epoch [1100/10000], Loss: 0.000024
Epoch [1200/10000], Loss: 0.000020
Epoch [1300/10000], Loss: 0.000018
Epoch [1400/10000], Loss: 0.000016
Epoch [1500/10000], Loss: 0.000015
Epoch [1600/10000], Loss: 0.000014
Epoch [1700/10000], Loss: 0.000013
Epoch [1800/10000], Loss: 0.000012
Epoch [1900/10000], Loss: 0.000012
Epoch [2000/10000], Loss: 0.000011
Epoch [2100/10000], Loss: 0.000010
Epoch [2200/10000], Loss: 0.000010
Epoch [2300/10000], Loss: 0.000009
Epoch [2400/10000], Loss: 0.000009
Epoch [2500/10000], Loss: 0.000008
Epoch [2600/10000], Loss: 0.000008
Epoch [2700/10000], Loss: 0.000008
Epoch [2800/10000], Loss: 0.000007
Epoch [2900/10000], Loss: 0.0

In [34]:
b1_alpha_2, weights_w1_alpha_2, b2_alpha_2, weights_w2_alpha_2 = extract_model_params(model_alpha_2)

In [35]:
integral_alpha_2 = get_NN_analytical_integral(alpha, beta, b1_alpha_2, b2_alpha_2, weights_w1_alpha_2, weights_w2_alpha_2)

In [36]:
print(f"The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: {float(integral_alpha_2):.6f}")

The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: 0.499994


In [37]:
true_integral_alpha_2 = (1/2)

In [38]:
print(f"The true cosinus integral is: {float(true_integral_alpha_2):.6f}")

The true cosinus integral is: 0.500000


## Integration of function 1.1.3:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 1, b = 0 $

In [48]:
a = 1
b = 0

In [49]:
model_alpha_3 = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha_3.parameters(), lr=learning_rate)

In [50]:
x_train_alpha_3, y_train_alpha_3 = generate_data(alpha_func, 10000)

In [51]:
train_model(model_alpha_3, criterion, optimizer_alpha, x_train_alpha_3, y_train_alpha_3, num_epochs)

Epoch [100/10000], Loss: 0.000017
Epoch [200/10000], Loss: 0.000013
Epoch [300/10000], Loss: 0.000010
Epoch [400/10000], Loss: 0.000007
Epoch [500/10000], Loss: 0.000005
Epoch [600/10000], Loss: 0.000004
Epoch [700/10000], Loss: 0.000004
Epoch [800/10000], Loss: 0.000003
Epoch [900/10000], Loss: 0.000003
Epoch [1000/10000], Loss: 0.000003
Epoch [1100/10000], Loss: 0.000003
Epoch [1200/10000], Loss: 0.000003
Epoch [1300/10000], Loss: 0.000003
Epoch [1400/10000], Loss: 0.000003
Epoch [1500/10000], Loss: 0.000003
Epoch [1600/10000], Loss: 0.000004
Epoch [1700/10000], Loss: 0.000003
Epoch [1800/10000], Loss: 0.000015
Epoch [1900/10000], Loss: 0.000003
Epoch [2000/10000], Loss: 0.000003
Epoch [2100/10000], Loss: 0.000016
Epoch [2200/10000], Loss: 0.000003
Epoch [2300/10000], Loss: 0.000002
Epoch [2400/10000], Loss: 0.000002
Epoch [2500/10000], Loss: 0.000002
Epoch [2600/10000], Loss: 0.000002
Epoch [2700/10000], Loss: 0.000002
Epoch [2800/10000], Loss: 0.000002
Epoch [2900/10000], Loss: 0.0

In [52]:
b1_alpha_3, weights_w1_alpha_3, b2_alpha_3, weights_w2_alpha_3 = extract_model_params(model_alpha_3)

In [53]:
integral_alpha_3 = get_NN_analytical_integral(alpha, beta, b1_alpha_3, b2_alpha_3, weights_w1_alpha_3, weights_w2_alpha_3)

In [54]:
print(f"The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: {float(integral_alpha_3):.6f}")

The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: 0.500023


In [73]:
true_integral_alpha_3 = (1/2)

In [74]:
print(f"The true cosinus integral is: {float(true_integral_alpha_3):.6f}")

The true cosinus integral is: 0.500000
